In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
from PIL import Image
import random
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc, roc_auc_score

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# TensorFlow and Keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.regularizers import l2
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils 

# import keras
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras import optimizers 

print(tf.__version__)



2.1.0


In [22]:
roc_mobilenetv2_SGD_input2 = pd.read_csv('/home/jovyan/scratch-shared/Linnea/Join channels differently/MobileNetV2/SGD/lr=0.01/rocauc_mobilenet_Ph__pretrained_SGD_lr0.01_batch32_may18.csv')
roc_mobilenetv2_Adam_input2 = pd.read_csv('/home/jovyan/scratch-shared/Linnea/Join channels differently/MobileNetV2/Adam/rocauc_mobilenet_Ph__pretrained_Adam_lr0.0001_batch32_may17.csv')
roc_mobilenetv2_np_input2 = pd.read_csv('/home/jovyan/scratch-shared/Linnea/Join channels differently/MobileNetV2/Not pretrained/rocauc_mobilenet_Ph__notpretrained_Adam_lr0.0001_batch32_may17.csv')



In [17]:
roc_mobilenetv2_SGD_input2

,Unnamed: 0,target/activity,roc area
0,0,ALK,0.683936
1,1,AMPK,0.674577
2,2,ATM,0.646680
3,3,ATM/ATR,0.534081
4,4,Akt,0.655940
...,...,...,...
76,109,control,0.708636
77,110,mTOR,0.690808
78,111,"mTOR,PI3K",0.618254
79,112,"mTOR,PI3K,Akt",0.825167


In [28]:

sorted_df_sgd = roc_mobilenetv2_SGD_input2.sort_values(by='roc area', ascending=True)
sorted_df_sgd

,Unnamed: 0,target/activity,roc area
47,65,"PI3K,mTOR",0.375308
39,55,PAK,0.377446
20,29,FAK,0.398617
15,21,"DNA-PK,PI3K,mTOR",0.404685
72,103,"c-Met,cKit,PDGFR,Flt,c-RET",0.414864
...,...,...,...
25,35,"Flt,Bcr-Abl,AuroraKinase",0.776930
13,18,CHK,0.803168
79,112,"mTOR,PI3K,Akt",0.825167
8,10,"AuroraKinase,FGFR,Bcr-Abl,c-RET,Src",0.826863


In [20]:
sorted_df_adam = roc_mobilenetv2_Adam_input2.sort_values(by='roc area', ascending=True)
sorted_df_adam

,Unnamed: 0,target/activity,roc area
35,50,"JAK,EGFR",0.604349
34,49,JAK,0.680467
1,1,AMPK,0.718701
33,48,I¦ÊB/IKK,0.751272
66,94,"VEGrFR,Src,Raf,Bcr-Abl",0.761669
...,...,...,...
19,28,ERK,0.999286
24,34,"Flt,AuroraKinase,VEGrFR",0.999509
13,18,CHK,0.999911
73,105,"cKit,FGFR,Flt,VEGrFR,PDGFR",1.000000


In [23]:
sorted_df_np = roc_mobilenetv2_np_input2.sort_values(by='roc area', ascending=True)
sorted_df_np

,Unnamed: 0,target/activity,roc area
42,58,"PDGFR,cKit,Bcr-Abl",0.025908
24,34,"Flt,AuroraKinase,VEGrFR",0.059170
41,57,"PDGFR,VEGrFR",0.059750
67,96,"VEGrFR,cKit,Flt",0.062651
49,68,PLK,0.102393
...,...,...,...
9,12,BTK,0.779681
55,76,S1PReceptor,0.799911
74,107,"cKit,Raf,VEGrFR",0.847724
25,35,"Flt,Bcr-Abl,AuroraKinase",0.886256


In [31]:
type(sorted_df_np)

pandas.core.frame.DataFrame

In [39]:
joined_dfs = pd.concat([sorted_df_sgd, sorted_df_adam,sorted_df_np], axis=1, sort=False)
joined_dfs

,Unnamed: 0,target/activity,roc area,Unnamed: 0,target/activity,roc area,Unnamed: 0,target/activity,roc area
0,0,ALK,0.683936,0,ALK,0.929451,0,ALK,0.320303
1,1,AMPK,0.674577,1,AMPK,0.718701,1,AMPK,0.498112
2,2,ATM,0.646680,2,ATM,0.923139,2,ATM,0.582172
3,3,ATM/ATR,0.534081,3,ATM/ATR,0.971763,3,ATM/ATR,0.653612
4,4,Akt,0.655940,4,Akt,0.912251,4,Akt,0.476435
...,...,...,...,...,...,...,...,...,...
76,109,control,0.708636,109,control,0.797087,109,control,0.321163
77,110,mTOR,0.690808,110,mTOR,0.904369,110,mTOR,0.532020
78,111,"mTOR,PI3K",0.618254,111,"mTOR,PI3K",0.925523,111,"mTOR,PI3K",0.127055
79,112,"mTOR,PI3K,Akt",0.825167,112,"mTOR,PI3K,Akt",0.994110,112,"mTOR,PI3K,Akt",0.434360
